In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
import xlrd

In [2]:
def convert_datetime(a):
    if pd.isnull(a):
        return pd.NaT
    
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S",
                    "%d-%B"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return
def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.xlsx'):

        df=pd.concat(pd.read_excel(filename,sheet_name=None, skiprows=0))
        list_files.append(df)

    df1 = pd.concat(list_files,   ignore_index=True)
    
    return df1
def input_dataraw( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*sheet1.csv'):
        list_files.append(filename)

    df1 = pd.concat(map(pd.read_csv, list_files),   ignore_index=True)
    
    return df1
def input_data_2_format(data_dir):
    lst=[]
    lst_file = glob.glob("data_dir/*False_Rock_d*")
    for f in lst_file:
        try:
            data = pd.read_csv(f,index_col=0)
            lst.append(data)
        except:
            data = pd.read_excel(f,index_col=0)
            lst.append(data)
    df1=pd.concat(lst,ignore_index=True)
    return df1
def merge_csv_files(folder_path, column_names):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, dtype={'BPO QA Task ID': object, 'object id': object, 'Queue ID': object})
            df['Week'] = os.path.basename(file_path).replace('.csv', '')
            df = df[column_names]
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")

    merged_df = pd.concat(dfs, axis=0, ignore_index=True)
    return merged_df

def merge_csv_files_upl(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path)
            df['source_file'] = os.path.basename(file_path)
            df.columns = df.columns.str.lower()
            df.columns = df.columns.map(lambda x: re.sub(r'\s+', ' ', x.strip()))
            df.columns = df.columns.str.replace(' ', '_')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df

In [3]:
def rename_csv_files(folder_path, resolve_time_column, sheet_name=None):
    files = os.listdir(folder_path)
    
    for file in files:
         if file.endswith(".csv") and "Output" not in file:
            file_path = os.path.join(folder_path, file)
            if sheet_name is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.read_csv(file_path, sheet_name=sheet_name)
                
            first_title = 'Output QA Queues'
            
            df[resolve_time_column] = pd.to_datetime(df[resolve_time_column])
            
            start_date = df[resolve_time_column].min()
            end_date = df[resolve_time_column].max()
            
            first_title = re.sub(r'[\\/:"*?<>|]+', '', first_title)
            
            start_date_str = start_date.strftime("%Y-%m-%d")
            end_date_str = end_date.strftime("%Y-%m-%d")
            
            new_file_name = f"{first_title}_{start_date_str}_{end_date_str}.csv"
            new_file_path = os.path.join(folder_path, new_file_name)
            
            if os.path.exists(new_file_path):
                os.remove(new_file_path)
            
            os.rename(file_path, new_file_path)

dir = 'D:/Data clean/All Queues - Diff case Rawdata/Sample size - QA queues'
try:
    rename_csv_files(dir,'Date')
except: 
    pass

In [4]:
# Supporting 

#Queue
queue_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Queue List',dtype={'MOD QUEUE ID': object,'QA QUEUE ID':object,'Queue _ID':object})
queue_list = pd.DataFrame(queue_list)
queue_list = queue_list.dropna(subset=['QA QUEUE NAME'])

#Week
week_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Week')
month_mapping = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}
week_list['Month'] = week_list['Month'].map(month_mapping)

#Policy Errors
policy_errors_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Policy Errors').values
policy_errors_list = [item for sublist in policy_errors_list for item in sublist]

In [5]:
team = {
    "Regular":['Livestream', 'IPR', 'Seller', 'Anchor Video','CB Product','LL Product'],
    "Appeal":['Appeal'],
    "Side Project":['Side Project'],
    "E-UPL":['E-UPL'],
    "G-UPL":['G-UPL'],
}

aliases_final_decision = {"Failed appeal":['failed appeal','mod wrong','mods wrong','qa wrong'] ,
            "Edge case":['edge case', 'borderline','boderline'],
            "Appeal successfully":['appeal successfully','qa wrong','qas wrong','qa correct','qa corect'],
            }

In [6]:
week_filter = {'W43','W44','W45','W46'}

'W1','W2','W3','W4','W5','W6','W7','W8','W9','W10','W11','W12','W13','W14','W15','W16','W17','W18','W19','W20','W21','W22','W23','W24','W25','W26','W27','W28','W29','W30','W31','W32','W33','W34','W35','W36','W37','W38','W39'

In [7]:
queue_list

,BUSINESS,COMPOUND,CATEGORY,MOD QUEUE ID,MOD QUEUE NAME,QA QUEUE ID,QA QUEUE NAME,QUEUE GROUP,STATUS,Priority,Latency,AHT (secs),Policy_type,Classify
0,Regular,CB Product,Product,7138022488281285122,VN CB Product High VV 02,7141278281147417090,[QA] VN CB Product High VV 02,HCM_EC_CB_Product,Active,High,<1 hr,40.0,Multi-choice,QA moderated
1,Regular,CB Product,Product,7072601148343009794,VN CB Product Initial Review,7141277606208438786,[QA] VN CB Product Initial Review 02,HCM_EC_CB_Product,Active,High,<1 hr,40.0,Multi-choice,QA moderated
2,Regular,CB Product,Product,7138022488281301506,VN CB Product Report 02,7141277606208455170,[QA] VN CB Product Report 02,HCM_EC_CB_Product,Active,High,<1 hr,40.0,Multi-choice,QA moderated
3,Regular,Anchor Video,Short Video,7138022488281252354,VN LL Anchor Video 02,7141277606208373250,[QA] VN LL Anchor Video 02,HCM_EC_Anchor,Active,High,<10mins,42.0,Single-choice,QA moderated
4,Regular,Anchor Video,Short Video,7138155829152203265,VN LL Anchor Video High VV 02,7141104196501258754,[QA] VN LL Anchor Video High VV 02,HCM_EC_Anchor,Active,High,<10mins,42.0,Single-choice,QA moderated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Labeling,Labeling,Labeling,NaN,TC_HCM_MOD_MODEL LABELING_IPR,NaN,TC_HCM_MOD_MODEL LABELING_IPR,NaN,NaN,NaN,NaN,NaN,Multi-choice,QA moderated
82,Labeling,Labeling,Labeling,NaN,TC_HCM_MOD_MODEL LABELING_PRODUCT,NaN,TC_HCM_MOD_MODEL LABELING_PRODUCT,NaN,NaN,NaN,NaN,NaN,Multi-choice,QA moderated
83,Labeling,Labeling,Labeling,NaN,verifier_1,NaN,verifier_1,NaN,NaN,NaN,NaN,NaN,Multi-choice,QA moderated
84,Labeling,Labeling,Labeling,NaN,verifier_2,NaN,verifier_2,NaN,NaN,NaN,NaN,NaN,Multi-choice,QA moderated


In [8]:
#Completion Rate
CR_rawdata_folderpath = 'Sample size - QA queues'
CR_rawdata_columns = ['Date',	'Queue Title',	'Queue ID', 'No. of Input','Avg. Handling Time(s)', 'No. of Output']
CR_rawdata_noaction = merge_csv_files(CR_rawdata_folderpath,CR_rawdata_columns)
CR_rawdata_noaction['Date'] = pd.to_datetime(CR_rawdata_noaction['Date'].apply(convert_datetime))
CR_rawdata = CR_rawdata_noaction[['Date',	'Queue Title',	'Queue ID',	'No. of Input','No. of Output']].rename(columns={'ï»¿Date':'Date'})
CR_rawdata = CR_rawdata.sort_values(by='No. of Output',ascending=True)
CR_rawdata = CR_rawdata.drop_duplicates(subset=['Date','Queue Title','Queue ID'],keep='last')
CR_rawdata['Queue Title'] = CR_rawdata['Queue Title'].str.replace('  ', ' ').str.replace('QA 【UG-2R-GnE】TTS-vi-VN', 'QA [UG-2R-GnE] TTS-vi-VN')
idx = CR_rawdata.groupby(['Date', 'Queue Title', 'Queue ID'])['No. of Output'].idxmax()
CR_rawdata = CR_rawdata.loc[idx]
CR_rawdata = pd.merge(CR_rawdata, queue_list, how='left', left_on='Queue Title', right_on='QA QUEUE NAME')
CR_1 = CR_rawdata[CR_rawdata['COMPOUND'].isin(['E-UPL','G-UPL'])]
CR_1 = pd.merge(CR_1, week_list, how='left', left_on='Date', right_on='Date')
CR_2 = CR_rawdata[~CR_rawdata['COMPOUND'].isin(['E-UPL','G-UPL'])]
CR_2 = pd.merge(CR_2, week_list, how='left', left_on='Date', right_on='QA_Date')
CR_rawdata = pd.concat([CR_1, CR_2])
CR_rawdata['Date'] = CR_rawdata['Date'].fillna(CR_rawdata['Date_x'])
CR_rawdata['Week'] = np.where(CR_rawdata['COMPOUND'].str.contains('UPL'), CR_rawdata['UPL_Week'], CR_rawdata['Mod_Week'])
CR_rawdata = CR_rawdata[['Quarter','Month','Week','Date', 'COMPOUND','MOD QUEUE NAME','QA QUEUE NAME', 'No. of Input', 'No. of Output']]
CR = CR_rawdata.groupby(by=['Quarter','Month','Week','Date','COMPOUND','QA QUEUE NAME'],as_index=False).agg({'No. of Input':'sum','No. of Output':'sum'}).rename(columns={'No. of Input':'Input','No. of Output':'Output'})
CR = CR[CR['Output']!= 0]
CR = CR[CR['Week'].str.contains('|'.join(week_filter))]
CR['%CR'] = CR['Output']/CR['Input']
CR_Week = CR_rawdata.groupby(by=['Quarter','Month','Week','COMPOUND','QA QUEUE NAME'],as_index=False).agg({'No. of Input':'sum','No. of Output':'sum'}).rename(columns={'No. of Input':'Input','No. of Output':'Output'})

In [9]:
aht_qa = CR_rawdata_noaction[(CR_rawdata_noaction['Date'].dt.strftime('%Y-%m-%d') >= '2023-08-01') & (CR_rawdata_noaction['Date'].dt.strftime('%Y-%m-%d') <= '2023-10-31')][['Date','Queue Title',	'Queue ID','Avg. Handling Time(s)']]
aht_qa['Month'] = aht_qa['Date'].dt.month
aht_qa.insert(0, 'Month', aht_qa.pop('Month'))
aht_qa = aht_qa.groupby(by=['Month','Queue Title'], as_index=False).agg({'Avg. Handling Time(s)':'mean'})
aht_qa['Avg. Handling Time(s)'] = aht_qa['Avg. Handling Time(s)'].round(2)
aht_qa.to_excel('aht_qa.xlsx')

In [10]:
CR_rawdata
CR_category = CR_rawdata.groupby(by=['Week','COMPOUND'],as_index=False).agg({'No. of Input':'sum','No. of Output':'sum'}).rename(columns={'No. of Input':'Input','No. of Output':'Output'})
for k, v in team.items():
    pat = '|'.join(v)
    CR_category.loc[CR_category['COMPOUND'].str.contains(pat, case=False),'Team'] = k
CR_category = CR_category.groupby(by=['Week','Team'],as_index=False).agg({'Input':'sum','Output':'sum'})
CR_category = CR_category[CR_category['Week'].str.contains('|'.join(week_filter))]
CR_category['%CR'] = CR_category['Output']/CR_category['Input']
CR_category['Week'].unique()

C:\Users\v6210227\AppData\Local\Temp\ipykernel_3724\840782199.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Regular' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  CR_category.loc[CR_category['COMPOUND'].str.contains(pat, case=False),'Team'] = k


array(['W43 (10.22 - 10.28)', 'W43 (10.26 - 11.01)',
       'W44 (10.29 - 11.04)', 'W44 (11.02 - 11.08)',
       'W45 (11.05 - 11.11)', 'W45 (11.09 - 11.15)',
       'W46 (11.12 - 11.18)'], dtype=object)

In [11]:
#Diff case
regular_appeal_folderpath = 'Regular_appeal'
regular_appeal_columns = ['Week','Queue Name', 'Queue ID', 'mod', 'TL', 'mod_date',
       'object id', 'mod_result', 'mod_reason', 'mod_aht', 'BPO QA',
       'BPO QA Date', 'BPO QA Task ID', 'BPO QA Link', 'BPO QA Result',
       'BPO QA Reason', 'Diff case', 'Final Decision', 'Final False Error',
       'Policy Error', 'Error Category 1', 'Error Category 2','Error Category 3']
regular_appeal_Rawdata = merge_csv_files(regular_appeal_folderpath,regular_appeal_columns)
regular_appeal_Rawdata = regular_appeal_Rawdata[~regular_appeal_Rawdata['Final Decision'].isnull()]
regular_appeal_Rawdata['BPO QA Date'] = pd.to_datetime(regular_appeal_Rawdata['BPO QA Date'].apply(convert_datetime))
regular_appeal_Rawdata['Final Decision'] = regular_appeal_Rawdata['Final Decision'].fillna('Mod correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    regular_appeal_Rawdata.loc[regular_appeal_Rawdata['Final Decision'].str.contains(pat, case=False),'Final Decision'] = k

Error reading file Regular_appeal\W43 (10.22 - 10.28).csv: "['TL'] not in index"
Error reading file Regular_appeal\W44 (10.29 - 11.04).csv: "['TL'] not in index"
Error reading file Regular_appeal\W45 (11.05 - 11.11).csv: "['TL'] not in index"
Error reading file Regular_appeal\W46 (11.12 - 11.18).csv: "['TL'] not in index"
Error reading file Regular_appeal\W47 (11.19 - 11.25).csv: "['TL'] not in index"
Error reading file Regular_appeal\W48 (11.26 - 12.02).csv: "['TL'] not in index"


In [12]:
regular_appeal_Rawdata = regular_appeal_Rawdata.loc[:, ~regular_appeal_Rawdata.columns.str.contains('^Unnamed')]

In [13]:
regular_appeal_Rawdata = regular_appeal_Rawdata.sort_values(by=['BPO QA Date'],ascending=True)

In [14]:
# Regular/Appeal - Queues (Mod Accr)

Diff_case = regular_appeal_Rawdata[['Week','Queue Name','mod','BPO QA Task ID','mod_date','BPO QA Date','Final Decision']]
Mod_Accr = Diff_case.groupby(['Week', 'BPO QA Date','Queue Name'], as_index=False).apply(lambda x: pd.Series({
    'Edge case': (x['Final Decision'] == 'Edge case').sum(),
    'Failed appeal': (x['Final Decision'] == 'Failed appeal').sum()
}))

Mod_Accr = pd.merge(CR,Mod_Accr,how='left',left_on=['Week','Date','QA QUEUE NAME'],right_on=['Week','BPO QA Date','Queue Name'])
compound_filter = ['Anchor Video','Livestream','CB Product','LL Product','Seller','IPR','Appeal']
Mod_Accr = Mod_Accr[Mod_Accr['COMPOUND'].isin(compound_filter)]

Mod_Accr = Mod_Accr.fillna(0)

Final_Mod_Accr = Mod_Accr[['Quarter','Month','Week','Date','COMPOUND','QA QUEUE NAME','Output','Failed appeal','Edge case']]
Final_Mod_Accr = Final_Mod_Accr.rename(columns={'Out':'Sample Size'})
Final_Mod_Accr['%Accr'] = 1-(Final_Mod_Accr['Failed appeal'] / Final_Mod_Accr['Output'])
Final_Mod_Accr['%Overall_Accr'] = 1-((Final_Mod_Accr['Edge case']+Final_Mod_Accr['Failed appeal']) / Final_Mod_Accr['Output'])
Final_Mod_Accr = Final_Mod_Accr.dropna(subset=['%Accr'])
Final_Mod_Accr = Final_Mod_Accr[Final_Mod_Accr['Week'].str.contains('|'.join(week_filter))]

In [15]:
CR_FN_FP = CR.groupby(by=['Quarter','Month','Week','COMPOUND','QA QUEUE NAME'], as_index = False).agg({'Output':'sum'})
CR_FN_FP

,Quarter,Month,Week,COMPOUND,QA QUEUE NAME,Output
0,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Recommendation L2L Sample QA,69
1,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search CB Sample QA,1
2,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,121
3,4.0,Nov,W43 (10.26 - 11.01),G-UPL,[VN] UP Livestream Sample QA,357
4,4.0,Nov,W43 (10.26 - 11.01),G-UPL,[VN] UP Video Sample QA,917
...,...,...,...,...,...,...
213,4.0,Oct,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search CB Sample QA,19
214,4.0,Oct,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,151
215,4.0,Oct,W43 (10.26 - 11.01),G-UPL,[VN] UP Livestream Sample QA,452
216,4.0,Oct,W43 (10.26 - 11.01),G-UPL,[VN] UP Product CB Sample QA,14


In [16]:
#QA ACCR

qa_dtset_folderpath = 'QA_Diff_Cases'
qa_dtset_columns = ['Week','2nd Round Date','Queue Name','1st round Name','Object ID','BD QA Decision']
qa_dtset = merge_csv_files(qa_dtset_folderpath,qa_dtset_columns)
qa_dtset['BD QA Decision'] = qa_dtset['BD QA Decision'].fillna('QA Correct')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    qa_dtset.loc[qa_dtset['BD QA Decision'].str.contains(pat, case=False),'BD QA Decision'] = k

qa_dtset['2nd Round Date'] = pd.to_datetime(qa_dtset['2nd Round Date'].apply(convert_datetime))
qa_dtset['2nd Round Date'].sort_values(ascending=True)
qa_dtset = pd.merge(qa_dtset,week_list,how='left',left_on='2nd Round Date',right_on='Date')

qa_dtset['1st round Name'] = qa_dtset['1st round Name'].replace('robot_estimate@','',regex=True)

qa_correct = qa_dtset.groupby(['Quarter','Month','Week','1st round Name'],as_index=False)['BD QA Decision'].apply(lambda x: x[(x == 'QA Correct') | (x == 'Appeal successfully') | (x == 'Edge case') | (x.isna())].count())
qa_sampling = qa_dtset.groupby(['Quarter','Month','Week','1st round Name'],as_index=False)['Queue Name'].apply(lambda y: y[(y != '')].count())
final_qa_accr = pd.merge(qa_sampling,qa_correct,how='inner',on=['Quarter','Month','Week','1st round Name']).rename(columns={'Queue Name':'Sampling','BD QA Decision':'QA Correct'})
final_qa_accr['%Accr'] = round(final_qa_accr['QA Correct'] / final_qa_accr['Sampling'],4)
final_qa_accr

,Quarter,Month,Week,1st round Name,Sampling,QA Correct,%Accr
0,1.0,Feb,W5 (01.28 - 02.03),chinh.nh1@trans-cosmos.com.vn,4,4,1.0000
1,1.0,Feb,W5 (01.28 - 02.03),dieu.mtb@trans-cosmos.com.vn,10,9,0.9000
2,1.0,Feb,W5 (01.28 - 02.03),duy.nt2@trans-cosmos.com.vn,50,50,1.0000
3,1.0,Feb,W5 (01.28 - 02.03),hao.ltn@trans-cosmos.com.vn,63,62,0.9841
4,1.0,Feb,W5 (01.28 - 02.03),hien.pt@trans-cosmos.com.vn,44,42,0.9545
...,...,...,...,...,...,...,...
1325,4.0,Oct,W43 (10.24 - 10.30),tram.nn@trans-cosmos.com.vn,4,4,1.0000
1326,4.0,Oct,W43 (10.24 - 10.30),tu.mha@trans-cosmos.com.vn,21,21,1.0000
1327,4.0,Oct,W43 (10.24 - 10.30),uyen.hnp1@trans-cosmos.com.vn,8,8,1.0000
1328,4.0,Oct,W43 (10.24 - 10.30),vy.tnt@trans-cosmos.com.vn,5,5,1.0000


In [17]:
#General UPL Diff case
G_UPL_rawdata = merge_csv_files_upl('G_UPL')
G_UPL_rawdata = G_UPL_rawdata.dropna(subset='queue_name')
G_UPL_rawdata['1st_date'] = pd.to_datetime(G_UPL_rawdata['1st_date'].apply(convert_datetime))            
G_UPL_rawdata['2nd_date'] = pd.to_datetime(G_UPL_rawdata['2nd_date'].apply(convert_datetime))
G_UPL_rawdata = pd.merge(G_UPL_rawdata,week_list,how='left',left_on='2nd_date',right_on='Date').rename(columns={'UPL_Week':'Week'})
G_UPL_rawdata.sort_values(by=['2nd_date'],ascending=True)
G_UPL_rawdata['bd_qa_decision'] = G_UPL_rawdata['bd_qa_decision'].fillna('')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    G_UPL_rawdata.loc[G_UPL_rawdata['bd_qa_decision'].str.contains(pat, case=False),'final_result'] = k
def set_policy_error(row):
    if row['final_result'] in ['Failed appeal', 'Edge case']:
        return row['2nd_rejection_reason']
    else:
        return row['1st_rejection_reason']
G_UPL_rawdata['policy_error'] = G_UPL_rawdata.apply(set_policy_error, axis=1)
G_UPL_false_case = G_UPL_rawdata.copy()
G_UPL_false_case['final_result_(copy)'] = G_UPL_false_case['final_result']
G_UPL_false_case = G_UPL_false_case.groupby(by=['Week','Date','queue_name'], as_index=False)\
                         .agg({'final_result': lambda x: x[x.eq('Failed appeal')].count() if 'Failed appeal' in x.values else 0,
                               'final_result_(copy)': lambda x: x[x.eq('Edge case')].count() if 'Edge case' in x.values else 0})\
                         .rename(columns={'queue_name': 'Queue Name', 'final_result': 'Failed appeal', 'final_result_(copy)': 'Edge case'})
G_UPL_false_case = G_UPL_false_case[['Week','Date','Queue Name','Failed appeal','Edge case']]
Final_G_UPL = pd.merge(CR,G_UPL_false_case,how='left',left_on=['Week','Date','QA QUEUE NAME'],right_on=['Week','Date','Queue Name'])
Final_G_UPL = Final_G_UPL[Final_G_UPL['COMPOUND']=="G-UPL"][['Quarter','Month','Week','Date', 'COMPOUND', 'QA QUEUE NAME','Output', 'Failed appeal','Edge case']]
Final_G_UPL = Final_G_UPL.groupby(by=['Quarter','Month','Week', 'Date','COMPOUND', 'QA QUEUE NAME'],as_index=False).agg({'Output':'sum', 'Failed appeal':'sum','Edge case':'sum'})
Final_G_UPL = Final_G_UPL[['Quarter','Month','Week','Date','COMPOUND','QA QUEUE NAME','Output','Failed appeal','Edge case']]
Final_G_UPL['%Accr'] = 1-(Final_G_UPL['Failed appeal'] / Final_G_UPL['Output'])
Final_G_UPL['%Overall_Accr'] = 1-((Final_G_UPL['Edge case']+Final_G_UPL['Failed appeal']) / Final_G_UPL['Output'])
Final_G_UPL = Final_G_UPL.dropna(subset=['%Accr'])
Final_G_UPL = Final_G_UPL[Final_G_UPL['Week'].str.contains('|'.join(week_filter))]

Error reading file G_UPL\W11 (03.13 - 03.19).csv: 'utf-8' codec can't decode byte 0xf3 in position 45160: invalid continuation byte
Error reading file G_UPL\W12 (03.20 - 03.26).csv: 'utf-8' codec can't decode byte 0xf3 in position 13702: invalid continuation byte
Error reading file G_UPL\W13 (03.27 - 04.02).csv: 'utf-8' codec can't decode byte 0xea in position 24332: invalid continuation byte
Error reading file G_UPL\W29 (07.17 - 07.23).csv: 'utf-8' codec can't decode byte 0xfd in position 73494: invalid start byte
Error reading file G_UPL\W30 (07.24 - 07.30).csv: 'utf-8' codec can't decode byte 0xe0 in position 51169: invalid continuation byte
Error reading file G_UPL\W31 (07.31 - 08.06).csv: 'utf-8' codec can't decode byte 0xf3 in position 31070: invalid continuation byte
Error reading file G_UPL\W32 (08.07 - 08.13).csv: 'utf-8' codec can't decode byte 0xf4 in position 26462: invalid continuation byte


C:\Users\v6210227\AppData\Local\Temp\ipykernel_3724\801090066.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Failed appeal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  G_UPL_rawdata.loc[G_UPL_rawdata['bd_qa_decision'].str.contains(pat, case=False),'final_result'] = k


In [18]:
#Shoping Mall UPL Diff case
E_UPL_rawdata = merge_csv_files_upl('E_UPL')
E_UPL_rawdata['1st_date'] = pd.to_datetime(E_UPL_rawdata['1st_date'].apply(convert_datetime))            
E_UPL_rawdata['2nd_date'] = pd.to_datetime(E_UPL_rawdata['2nd_date'].apply(convert_datetime))
E_UPL_rawdata = pd.merge(E_UPL_rawdata,week_list,how='left',left_on='2nd_date',right_on='Date').rename(columns={'UPL_Week':'Week'})
E_UPL_rawdata = E_UPL_rawdata[~E_UPL_rawdata['Week'].isnull()]
E_UPL_rawdata.sort_values(by=['2nd_date'],ascending=True)
E_UPL_rawdata['[bpo_qa]_decision'] = E_UPL_rawdata['[bpo_qa]_decision'].fillna('')
for k, v in aliases_final_decision.items():
    pat = '|'.join(v)
    E_UPL_rawdata.loc[E_UPL_rawdata['[bpo_qa]_decision'].str.contains(pat, case=False),'final_result'] = k
def set_policy_error(row):
    columns = ['1st_rejection_reason', '2nd_rejection_reason', '3rd_rejection_reason']
    reasons = row[columns].replace('', np.nan)
    last_valid = reasons.last_valid_index()
    return row[last_valid] if last_valid else np.nan

E_UPL_rawdata['policy_error'] = E_UPL_rawdata.apply(set_policy_error, axis=1)

E_UPL_rawdata_1 = E_UPL_rawdata.copy()
E_UPL_rawdata_1['final_result_(copy)'] = E_UPL_rawdata_1['final_result']
E_UPL_false_case = E_UPL_rawdata_1.groupby(by=['Week','Date','queue_name'], as_index=False)\
                         .agg({'final_result': lambda x: x[x.eq('Failed appeal')].count() if 'Failed appeal' in x.values else 0,
                               'final_result_(copy)': lambda x: x[x.eq('Edge case')].count() if 'Edge case' in x.values else 0})\
                         .rename(columns={'queue_name': 'Queue Name', 'final_result': 'Failed appeal', 'final_result_(copy)': 'Edge case'})
E_UPL_false_case = E_UPL_false_case[['Week','Date','Queue Name','Failed appeal','Edge case']]
Final_E_UPL = pd.merge(CR,E_UPL_false_case,how='left',left_on=['Week','Date','QA QUEUE NAME'],right_on=['Week','Date','Queue Name'])
Final_E_UPL = Final_E_UPL[Final_E_UPL['COMPOUND']=="E-UPL"][['Quarter','Month','Week','Date', 'COMPOUND', 'QA QUEUE NAME','Output', 'Failed appeal','Edge case']]
Final_E_UPL = Final_E_UPL.groupby(by=['Quarter','Month','Week', 'Date','COMPOUND', 'QA QUEUE NAME'],as_index=False).agg({'Output':'sum', 'Failed appeal':'sum','Edge case':'sum'})
Final_E_UPL = Final_E_UPL[['Quarter','Month','Week','Date','COMPOUND','QA QUEUE NAME','Output','Failed appeal','Edge case']]
Final_E_UPL['%Accr'] = 1-(Final_E_UPL['Failed appeal'] / Final_E_UPL['Output'])
Final_E_UPL['%Overall_Accr'] = 1-((Final_E_UPL['Edge case']+Final_E_UPL['Failed appeal']) / Final_E_UPL['Output'])
Final_E_UPL = Final_E_UPL.dropna(subset=['%Accr'])
Final_E_UPL = Final_E_UPL[Final_E_UPL['Week'].str.contains('|'.join(week_filter))]
Final_E_UPL

C:\Users\v6210227\AppData\Local\Temp\ipykernel_3724\182085927.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Failed appeal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  E_UPL_rawdata.loc[E_UPL_rawdata['[bpo_qa]_decision'].str.contains(pat, case=False),'final_result'] = k


,Quarter,Month,Week,Date,COMPOUND,QA QUEUE NAME,Output,Failed appeal,Edge case,%Accr,%Overall_Accr
0,4.0,Nov,W43 (10.26 - 11.01),2023-10-30,E-UPL,[VN] Mall UP Recommendation L2L Sample QA,56,1.0,0.0,0.982143,0.982143
1,4.0,Nov,W43 (10.26 - 11.01),2023-10-30,E-UPL,[VN] Mall UP Search CB Sample QA,1,0.0,0.0,1.000000,1.000000
2,4.0,Nov,W43 (10.26 - 11.01),2023-10-30,E-UPL,[VN] Mall UP Search L2L Sample QA,51,0.0,0.0,1.000000,1.000000
3,4.0,Nov,W43 (10.26 - 11.01),2023-10-31,E-UPL,[VN] Mall UP Recommendation L2L Sample QA,13,0.0,1.0,1.000000,0.923077
4,4.0,Nov,W43 (10.26 - 11.01),2023-10-31,E-UPL,[VN] Mall UP Search L2L Sample QA,69,2.0,0.0,0.971014,0.971014
5,4.0,Nov,W43 (10.26 - 11.01),2023-11-01,E-UPL,[VN] Mall UP Search L2L Sample QA,1,0.0,0.0,1.000000,1.000000
6,4.0,Nov,W44 (11.02 - 11.08),2023-11-02,E-UPL,[VN] Mall UP Recommendation CB Sample QA,27,0.0,0.0,1.000000,1.000000
7,4.0,Nov,W44 (11.02 - 11.08),2023-11-02,E-UPL,[VN] Mall UP Recommendation L2L Sample QA,36,0.0,0.0,1.000000,1.000000
8,4.0,Nov,W44 (11.02 - 11.08),2023-11-02,E-UPL,[VN] Mall UP Search CB Sample QA,1,0.0,0.0,1.000000,1.000000
9,4.0,Nov,W44 (11.02 - 11.08),2023-11-02,E-UPL,[VN] Mall UP Search L2L Sample QA,60,0.0,0.0,1.000000,1.000000


In [19]:
#Side project
Sideproject_rawdata_folderpath = 'Side_project'
Sideproject_rawdata_columns = ['Week','Queue Name','mod','BPO QA Task ID','mod_date','BPO QA Date','Final Decision','Final False Error','Policy Error','Error Category 1','Error Category 2', 'Error Category 3']
Sideproject_rawdata = merge_csv_files(Sideproject_rawdata_folderpath,Sideproject_rawdata_columns)
Sideproject_rawdata = Sideproject_rawdata.loc[:, ~Sideproject_rawdata.columns.str.contains('^Unnamed')]
Sideproject_rawdata['Queue Name'] = Sideproject_rawdata['Queue Name'].str.replace('QA 【UG-2R-GnE】TTS-vi-VN', 'QA [UG-2R-GnE] TTS-vi-VN')
Sideproject_rawdata['Queue Name'].unique()

Sideproject_rawdata['mod_date'] = pd.to_datetime(Sideproject_rawdata['mod_date'].apply(convert_datetime))            
Sideproject_rawdata['BPO QA Date'] = pd.to_datetime(Sideproject_rawdata['BPO QA Date'].apply(convert_datetime))

Diff_case_sideproject = Sideproject_rawdata[['Week','Queue Name','mod','BPO QA Task ID','mod_date','BPO QA Date','Final Decision']]

Mod_Accr_sideproject = Diff_case_sideproject.groupby(['Week','BPO QA Date', 'Queue Name'], as_index=False).apply(lambda x: pd.Series({
    'Edge case': (x['Final Decision'] == 'Edge case').sum(),
    'Failed appeal': (x['Final Decision'] == 'Failed appeal').sum()
}))
Mod_Accr_sideproject = pd.merge(CR,Mod_Accr_sideproject,how='left',left_on=['Week','Date','QA QUEUE NAME'],right_on=['Week','BPO QA Date','Queue Name'])
Mod_Accr_sideproject = Mod_Accr_sideproject[Mod_Accr_sideproject['COMPOUND']=='Side Project']
Mod_Accr_sideproject = Mod_Accr_sideproject.fillna(0)

Final_Mod_Accr_sideproject = Mod_Accr_sideproject[['Quarter','Month','Week','Date','COMPOUND','QA QUEUE NAME','Output','Failed appeal','Edge case']]
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject.rename(columns={'Out':'Sample Size'})
Final_Mod_Accr_sideproject['%Accr'] = 1-(Final_Mod_Accr_sideproject['Failed appeal'] / Final_Mod_Accr_sideproject['Output'])
Final_Mod_Accr_sideproject['%Overall_Accr'] = 1-((Final_Mod_Accr_sideproject['Edge case']+Final_Mod_Accr_sideproject['Failed appeal']) / Final_Mod_Accr_sideproject['Output'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject.dropna(subset=['%Accr'])
Final_Mod_Accr_sideproject = Final_Mod_Accr_sideproject[Final_Mod_Accr_sideproject['Week'].str.contains('|'.join(week_filter))]
Final_Mod_Accr_sideproject

,Quarter,Month,Week,Date,COMPOUND,QA QUEUE NAME,Output,Failed appeal,Edge case,%Accr,%Overall_Accr
50,4.0,Nov,W44 (10.29 - 11.04),2023-10-30,Side Project,QA [CB][VN]-Shoptab Product Image Dedicated,57,0.0,0.0,1.000000,1.000000
51,4.0,Nov,W44 (10.29 - 11.04),2023-10-30,Side Project,QA [L2L][VN]-Shoptab Product Image Dedicated,57,0.0,0.0,1.000000,1.000000
52,4.0,Nov,W44 (10.29 - 11.04),2023-10-30,Side Project,QA [VN] Seller Gov X Feige Chat,1,0.0,0.0,1.000000,1.000000
53,4.0,Nov,W44 (10.29 - 11.04),2023-10-30,Side Project,QA [VN] Shoptab Content cover image,90,1.0,0.0,0.988889,0.988889
98,4.0,Nov,W44 (10.29 - 11.04),2023-10-31,Side Project,QA [CB][VN]-Shoptab Product Image Dedicated,57,1.0,0.0,0.982456,0.982456
...,...,...,...,...,...,...,...,...,...,...,...
998,4.0,Oct,W43 (10.22 - 10.28),2023-10-28,Side Project,QA [VN] Shoptab Content cover image,90,0.0,0.0,1.000000,1.000000
1040,4.0,Oct,W43 (10.22 - 10.28),2023-10-29,Side Project,QA [CB][VN]-Shoptab Product Image Dedicated,57,0.0,0.0,1.000000,1.000000
1041,4.0,Oct,W43 (10.22 - 10.28),2023-10-29,Side Project,QA [L2L][VN]-Shoptab Product Image Dedicated,57,0.0,0.0,1.000000,1.000000
1042,4.0,Oct,W43 (10.22 - 10.28),2023-10-29,Side Project,QA [VN] Seller Gov X Feige Chat,95,0.0,0.0,1.000000,1.000000


In [20]:
#RCA - Accr Report
RCA_diff_dive_regular_appeal = regular_appeal_Rawdata.loc[(regular_appeal_Rawdata['Final Decision'] == 'Failed appeal') | (regular_appeal_Rawdata['Final Decision'] == 'Edge case'),['BPO QA Date','Queue Name','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']].rename(columns={'BPO QA Date':'Date','Queue Name':'QA QUEUE NAME'})
RCA_diff_dive_regular_sideproject = Sideproject_rawdata.loc[(Sideproject_rawdata['Final Decision'] == 'Failed appeal') | (Sideproject_rawdata['Final Decision'] == 'Edge case'),['BPO QA Date','Queue Name','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']].rename(columns={'BPO QA Date':'Date','Queue Name':'QA QUEUE NAME'})
RCA_diff_dive_e_upl = E_UPL_rawdata.loc[(E_UPL_rawdata['final_result']=='Failed appeal') | (E_UPL_rawdata['final_result']=='Edge case'),['2nd_date','queue_name','final_result','error_category_1','error_category_2', 'error_category_3']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','final_result':'Final Decision','error_category_1':'Error Category 1','error_category_2':'Error Category 2', 'error_category_3':'Error Category 3'})
RCA_diff_dive_g_upl = G_UPL_rawdata.loc[(G_UPL_rawdata['final_result']=='Failed appeal') | (G_UPL_rawdata['final_result']=='Edge case'),['2nd_date','queue_name','final_result','error_category_1','error_category_2', 'error_category_3']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','Final Result':'Final Decision','error_category_1':'Error Category 1','error_category_2':'Error Category 2', 'error_category_3':'Error Category 3'})
RCA_diff_dive_1 = pd.concat([RCA_diff_dive_regular_appeal.reset_index(drop=True),RCA_diff_dive_regular_sideproject.reset_index(drop=True)],axis=0)
RCA_diff_dive_2 = pd.concat([RCA_diff_dive_e_upl.reset_index(drop=True),RCA_diff_dive_g_upl.reset_index(drop=True)],axis=0)
RCA_diff_dive_1 = pd.merge(RCA_diff_dive_1,week_list,left_on='Date',right_on='QA_Date',how='left').rename(columns={'QA_Week':'Week'})
RCA_diff_dive_2 = pd.merge(RCA_diff_dive_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
RCA_diff_dive_1 = RCA_diff_dive_1[['Quarter','Month','Week','QA QUEUE NAME','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']]
RCA_diff_dive_2 = RCA_diff_dive_2[['Quarter','Month','Week','QA QUEUE NAME','Final Decision','Error Category 1', 'Error Category 2','Error Category 3']]
RCA_diff_dive =  pd.concat([RCA_diff_dive_1.reset_index(drop=True),RCA_diff_dive_2.reset_index(drop=True)],axis=0)
RCA_diff_dive = pd.merge(RCA_diff_dive,queue_list,how='left',left_on='QA QUEUE NAME',right_on='QA QUEUE NAME')
RCA_diff_dive_count = RCA_diff_dive.groupby(['Quarter','Month','Week', 'COMPOUND', 'MOD QUEUE NAME', 'QA QUEUE NAME', 'Final Decision', 'Error Category 1', 'Error Category 2', 'Error Category 3']).agg(
    quanity_failed_appeal=('Final Decision', lambda x: (x == 'Failed appeal').sum()),
    quanity_edge_case=('Final Decision', lambda x: (x == 'Edge case').sum())).reset_index()
RCA_diff_dive_count = RCA_diff_dive_count[RCA_diff_dive_count['Week'].str.contains('|'.join(week_filter))]
RCA_diff_dive_count

,Quarter,Month,Week,COMPOUND,MOD QUEUE NAME,QA QUEUE NAME,Final Decision,Error Category 1,Error Category 2,Error Category 3,quanity_failed_appeal,quanity_edge_case
5624,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Recommendation L2L Sample,[VN] Mall UP Recommendation L2L Sample QA,Edge case,2. SOP,2.3 Edge/Grey Cases,2.3.2 Visibility Issues,0,1
5625,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Recommendation L2L Sample,[VN] Mall UP Recommendation L2L Sample QA,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,1,0
5626,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample,[VN] Mall UP Search L2L Sample QA,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,1,0
5627,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample,[VN] Mall UP Search L2L Sample QA,Failed appeal,1. People,1.6 Product Knowledge Gap,1.6.1 Misinterpretation of the product,1,0
5628,4,Nov,W44 (10.29 - 11.04),Side Project,[CB][VN]-Shoptab Product Image Dedicated,QA [CB][VN]-Shoptab Product Image Dedicated,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,2,0
5629,4,Nov,W44 (10.29 - 11.04),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,Edge case,2. SOP,2.1 Policy Ambiguity,2.1.2 Unclear policy update/clarifications,0,1
5630,4,Nov,W44 (10.29 - 11.04),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,Edge case,2. SOP,2.1 Policy Ambiguity,2.1.3 Ambiguous policy wording,0,2
5631,4,Nov,W44 (10.29 - 11.04),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,Failed appeal,1. People,1.3 Negligence,1.3.3 Oversight,1,0
5632,4,Nov,W44 (10.29 - 11.04),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,Failed appeal,1. People,1.4 Content Misinterpretation,1.4.1 Misinterpretation of the content,12,0
5633,4,Nov,W44 (10.29 - 11.04),Side Project,[VN] Seller Gov X Feige Chat,QA [VN] Seller Gov X Feige Chat,Failed appeal,1. People,1.5 Policy Misinterpretation,1.5.1 Misinterpretation of the existing guideline,2,0


In [21]:
#Policy Errors - Accr Report
Policy_diff_dive_regular_appeal = regular_appeal_Rawdata.loc[(regular_appeal_Rawdata['Final Decision'] == 'Failed appeal') | (regular_appeal_Rawdata['Final Decision'] == 'Edge case'), ['BPO QA Date', 'Queue Name', 'Final Decision','Policy Error']].rename(columns={'BPO QA Date': 'Date', 'Queue Name': 'QA QUEUE NAME'})
Policy_diff_dive_sideproject = Sideproject_rawdata.loc[(Sideproject_rawdata['Final Decision'] == 'Failed appeal') | (Sideproject_rawdata['Final Decision'] == 'Edge case'), ['BPO QA Date', 'Queue Name', 'Final Decision','Policy Error']].rename(columns={'BPO QA Date': 'Date', 'Queue Name': 'QA QUEUE NAME'})
Policy_diff_dive_e_upl = E_UPL_rawdata.loc[(E_UPL_rawdata['final_result']=='Failed appeal') | (E_UPL_rawdata['final_result']=='Edge case'),['2nd_date','queue_name','final_result','policy_error']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','final_result':'Final Decision','policy_error':'Policy Error'})
Policy_diff_dive_g_upl = G_UPL_rawdata.loc[(G_UPL_rawdata['final_result']=='Failed appeal') | (G_UPL_rawdata['final_result']=='Edge case'),['2nd_date','queue_name','final_result','policy_error']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','final_result':'Final Decision','policy_error':'Policy Error'})
Policy_diff_dive_1 = pd.concat([Policy_diff_dive_regular_appeal.reset_index(drop=True),Policy_diff_dive_sideproject.reset_index(drop=True)],axis=0)
Policy_diff_dive_2 = pd.concat([Policy_diff_dive_e_upl.reset_index(drop=True),Policy_diff_dive_g_upl.reset_index(drop=True)],axis=0)
Policy_diff_dive_1 = pd.merge(Policy_diff_dive_1,week_list,left_on='Date',right_on='QA_Date',how='left').rename(columns={'QA_Week':'Week'})
Policy_diff_dive_1 = Policy_diff_dive_1[['Quarter','Month','Week','QA QUEUE NAME', 'Final Decision','Policy Error']]
Policy_diff_dive_2 = pd.merge(Policy_diff_dive_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
Policy_diff_dive_2 = Policy_diff_dive_2[['Quarter','Month','Week','QA QUEUE NAME', 'Final Decision','Policy Error']]
Policy_diff_dive = pd.concat([Policy_diff_dive_1.reset_index(drop=True), Policy_diff_dive_2.reset_index(drop=True)], axis = 0)

def split_rows(row):
    errors = str(row['Policy Error']).split(', ') if ', ' in str(row['Policy Error']) else str(row['Policy Error']).split(',')
    if len(errors) > 1:
        new_rows = [row.copy() for _ in range(len(errors))]
        for i, error in enumerate(errors):
            new_rows[i]['Policy Error'] = error.strip()
        return new_rows
    return [row]

Policy_diff_dive['Policy Error'] = Policy_diff_dive['Policy Error'].fillna('')

Policy_diff_dive = pd.concat([pd.DataFrame(split_rows(row)) for _, row in Policy_diff_dive.iterrows()], ignore_index=True)
Policy_diff_dive = Policy_diff_dive[['Quarter', 'Month', 'Week', 'QA QUEUE NAME', 'Final Decision', 'Policy Error']]

Policy_errors = policy_errors_list.copy()
Policy_diff_dive['Policy Error'] = Policy_diff_dive['Policy Error'].str.lower()
Policy_errors_pattern = "|".join(Policy_errors)
def pattern_searcher(search_str:str, search_list:str):
    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = np.nan
    return return_str
Policy_diff_dive['Policy Error'] = Policy_diff_dive['Policy Error'].astype(str).apply(lambda x: pattern_searcher(search_str=x, search_list=Policy_errors_pattern))
Policy_diff_dive = pd.merge(Policy_diff_dive,queue_list,how='left',left_on='QA QUEUE NAME',right_on='QA QUEUE NAME')
Policy_errors_count = Policy_diff_dive.groupby(['Quarter','Month','Week', 'COMPOUND', 'QA QUEUE NAME', 'Policy Error']).agg(quanity_failed_appeal=('Final Decision', lambda x: (x == 'Failed appeal').sum()), quanity_edge_case=('Final Decision', lambda x: (x == 'Edge case').sum())).reset_index()
Policy_errors_count['Policy Error'] = Policy_errors_count['Policy Error'].replace('incorrect category high risk', 'incorrect category-high risk')
Policy_errors_count = Policy_errors_count[Policy_errors_count['Week'].str.contains('|'.join(week_filter))]
Policy_errors_count

,Quarter,Month,Week,COMPOUND,QA QUEUE NAME,Policy Error,quanity_failed_appeal,quanity_edge_case
4250,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Recommendation L2L Sample QA,brand circumvention,1,1
4251,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,incomplete information,1,0
4252,4,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,inconsistent information,1,0
4253,4,Nov,W44 (10.29 - 11.04),Side Project,QA [CB][VN]-Shoptab Product Image Dedicated,shoptab ansa,1,0
4254,4,Nov,W44 (10.29 - 11.04),Side Project,QA [CB][VN]-Shoptab Product Image Dedicated,shoptab vulgar image,1,0
4255,4,Nov,W44 (10.29 - 11.04),Side Project,QA [VN] Seller Gov X Feige Chat,offering off-platform return refund to buyers,12,2
4256,4,Nov,W44 (10.29 - 11.04),Side Project,QA [VN] Seller Gov X Feige Chat,redirecting purchase outside platform,3,1
4257,4,Nov,W44 (10.29 - 11.04),Side Project,QA [VN] Shoptab Content cover image,nudity body exposure shoptab,1,0
4258,4,Nov,W44 (11.02 - 11.08),E-UPL,[VN] Mall UP Recommendation L2L Sample QA,misleading functionality and effect,1,0
4259,4,Nov,W44 (11.02 - 11.08),E-UPL,[VN] Mall UP Search L2L Sample QA,absolute terms,1,0


In [22]:
## FN/ FP Rate - Accr Report - Regular/Appeal
False_errors_regular_appeal = regular_appeal_Rawdata.loc[~regular_appeal_Rawdata['Final Decision'].isnull(),['BPO QA Date','Queue Name','Final Decision','Final False Error']].rename(columns={'BPO QA Date':'Date','Queue Name':'QA QUEUE NAME'})
False_errors_sideproject = Sideproject_rawdata.loc[~Sideproject_rawdata['Final Decision'].isnull(),['BPO QA Date','Queue Name','Final Decision','Final False Error']].rename(columns={'BPO QA Date':'Date','Queue Name':'QA QUEUE NAME'})
False_errors_e_upl = E_UPL_rawdata.loc[~E_UPL_rawdata['final_result'].isnull(),['2nd_date','queue_name','final_result','moderators_false_error']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','final_result':'Final Decision','moderators_false_error':'Final False Error'})
False_errors_g_upl = G_UPL_rawdata.loc[~G_UPL_rawdata['final_result'].isnull(),['2nd_date','queue_name','final_result','moderators_false_error']].rename(columns={'2nd_date':'Date','queue_name':'QA QUEUE NAME','final_result':'Final Decision','moderators_false_error':'Final False Error'})
False_errors_concat_1 = pd.concat([False_errors_regular_appeal.reset_index(drop=True), False_errors_sideproject.reset_index(drop=True)], axis = 0)
False_errors_concat_2 = pd.concat([False_errors_e_upl.reset_index(drop=True),False_errors_g_upl.reset_index(drop=True)], axis = 0)
False_errors_concat_1['Final False Error'] = False_errors_concat_1['Final False Error'].str.title()
False_errors_concat_2['Final False Error'] = False_errors_concat_2['Final False Error'].str.title()
False_errors_1 = pd.merge(False_errors_concat_1,week_list,left_on='Date',right_on='QA_Date',how='left').rename(columns={'QA_Week':'Week'})
False_errors_2 = pd.merge(False_errors_concat_2,week_list,left_on='Date',right_on='Date',how='left').rename(columns={'UPL_Week':'Week'})
False_errors_1 = False_errors_1[['Quarter','Month','Week','QA QUEUE NAME','Final Decision','Final False Error']]
False_errors_2 = False_errors_2[['Quarter','Month','Week','QA QUEUE NAME','Final Decision','Final False Error']]
False_errors =  pd.concat([False_errors_1.reset_index(drop=True), False_errors_2.reset_index(drop=True)], axis = 0)
Overkill =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal'])) & (False_errors['Final False Error'] == 'False Positive')]
Overkill_count = Overkill.groupby(['Quarter','Month','Week','QA QUEUE NAME']).size().reset_index(name='Overkill')
Overkill_Overall =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal','Edge case'])) & (False_errors['Final False Error'] == 'False Positive')]
Overkill_Overall_count = Overkill_Overall.groupby(['Quarter','Month','Week','QA QUEUE NAME']).size().reset_index(name='Overkill_Overall')
Leakage =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal'])) & (False_errors['Final False Error'] == 'False Negative')]
Leakage_count = Leakage.groupby(['Quarter','Month','Week','QA QUEUE NAME']).size().reset_index(name='Leakage')
Leakage_Overall =  False_errors[(False_errors['Final Decision'].isin(['Failed appeal','Edge case'])) & (False_errors['Final False Error'] == 'False Negative')]
Leakage_Overall_count = Leakage_Overall.groupby(['Quarter','Month','Week','QA QUEUE NAME']).size().reset_index(name='Leakage_Overall')
FP_false_errors = pd.merge(Overkill_Overall_count,Overkill_count,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','QA QUEUE NAME'])
FN_false_errors = pd.merge(Leakage_Overall_count,Leakage_count,how='left',left_on=['Week','QA QUEUE NAME'],right_on=['Week','QA QUEUE NAME'])
All_fn_fp = pd.merge(FP_false_errors,FN_false_errors,how='outer',on=['Week','QA QUEUE NAME'])
All_fn_fp = pd.merge(CR_FN_FP,All_fn_fp,how='left',on=['Week','QA QUEUE NAME'])
All_fn_fp=  pd.merge(All_fn_fp,queue_list,on=['COMPOUND','QA QUEUE NAME'],how='inner')
All_fn_fp = All_fn_fp[['Quarter','Month','Week','COMPOUND','MOD QUEUE NAME','QA QUEUE NAME','Overkill_Overall','Overkill','Leakage_Overall','Leakage','Output']]
All_fn_fp = All_fn_fp.sort_values(by=['Week'], ascending = True)
All_fn_fp = All_fn_fp[All_fn_fp['Week'].str.contains('|'.join(week_filter))]
All_fn_fp['%Overkill'] = All_fn_fp['Overkill'] / All_fn_fp['Output']
All_fn_fp['%Leakage'] = All_fn_fp['Leakage'] / All_fn_fp['Output']
All_fn_fp_cp = All_fn_fp.groupby(by=['Quarter','Month','Week',	'COMPOUND'], as_index = False).agg({
   'Overkill_Overall':'sum',
   'Overkill_Overall':'sum',
   'Overkill':'sum',
   'Leakage_Overall':'sum',
   'Leakage':'sum',
   'Output':'sum'})
All_fn_fp_cp['%Overkill'] = All_fn_fp_cp['Overkill'] / All_fn_fp_cp['Output']
All_fn_fp_cp['%Leakage'] = All_fn_fp_cp['Leakage'] / All_fn_fp_cp['Output']
All_fn_fp_cp

,Quarter,Month,Week,COMPOUND,Overkill_Overall,Overkill,Leakage_Overall,Leakage,Output,%Overkill,%Leakage
0,4.0,Nov,W43 (10.26 - 11.01),E-UPL,5.0,4.0,4.0,4.0,312,0.012821,0.012821
1,4.0,Nov,W43 (10.26 - 11.01),G-UPL,0.0,0.0,0.0,0.0,1274,0.000000,0.000000
2,4.0,Nov,W44 (10.29 - 11.04),Anchor Video,0.0,0.0,0.0,0.0,3977,0.000000,0.000000
3,4.0,Nov,W44 (10.29 - 11.04),Appeal,0.0,0.0,0.0,0.0,5947,0.000000,0.000000
4,4.0,Nov,W44 (10.29 - 11.04),CB Product,0.0,0.0,0.0,0.0,2621,0.000000,0.000000
5,4.0,Nov,W44 (10.29 - 11.04),IPR,0.0,0.0,0.0,0.0,5886,0.000000,0.000000
6,4.0,Nov,W44 (10.29 - 11.04),LL Product,0.0,0.0,0.0,0.0,6479,0.000000,0.000000
7,4.0,Nov,W44 (10.29 - 11.04),Labeling,0.0,0.0,0.0,0.0,1184,0.000000,0.000000
8,4.0,Nov,W44 (10.29 - 11.04),Livestream,0.0,0.0,0.0,0.0,2541,0.000000,0.000000
9,4.0,Nov,W44 (10.29 - 11.04),Seller,0.0,0.0,0.0,0.0,2421,0.000000,0.000000


In [23]:
#Labeling Sample size
labeling_rawdata = pd.read_excel('Labeling/Labeling accr.xlsx')
labeling_rawdata
labeling_sample_size = labeling_rawdata.groupby(['Date','Category'],as_index=False)['Sample size (QA output)'].apply(lambda y: y[y > 0].sum())
labeling_sample_size = labeling_sample_size[['Date','Category','Sample size (QA output)']].rename(columns={'Date':'Date',
                                                                                                            'Category':'QA QUEUE NAME',
                                                                                                            'Sample size (QA output)':'Output'}) 
labeling_mod_wrong = labeling_rawdata.groupby(['Date','Category'],as_index=False)['Mod wrong tagging'].apply(lambda y: y[y > 0].sum())
labeling_mod_wrong = labeling_mod_wrong[['Date','Category','Mod wrong tagging']].rename(columns={'Date':'Date',
                                                                                                            'Category':'QA QUEUE NAME',
                                                                                                            'Mod wrong tagging':'Failed appeal'})  
final_labeling = pd.merge(labeling_sample_size,labeling_mod_wrong,how='left',on=['Date','QA QUEUE NAME'])
final_labeling = pd.merge(final_labeling,week_list,how='left',on=['Date'])
final_labeling = pd.merge(final_labeling,queue_list,how='left',on=['QA QUEUE NAME'])
final_labeling = final_labeling[['Quarter','Month','Labeling_Week','Date','COMPOUND','QA QUEUE NAME','Output','Failed appeal']]
final_labeling = final_labeling.groupby(by=['Quarter','Month','Labeling_Week','Date','COMPOUND','QA QUEUE NAME'],as_index=False).agg({'Output':'sum','Failed appeal':'sum'}).rename(columns={'Labeling_Week':'Week'})
final_labeling['Edge case'] = 0
final_labeling['%Accr'] = 1-(final_labeling['Failed appeal'] / final_labeling['Output'])
final_labeling['%Overall_Accr'] = 1-((final_labeling['Edge case']+final_labeling['Failed appeal']) / final_labeling['Output'])
final_labeling = final_labeling.dropna(subset=['%Accr'])
final_labeling = final_labeling[final_labeling['Week'].str.contains('|'.join(week_filter))]

In [24]:
TOTAL_ACCR = pd.concat([Final_Mod_Accr, Final_E_UPL, Final_G_UPL, Final_Mod_Accr_sideproject, final_labeling], axis=0)
TOTAL_ACCR_WEEK = TOTAL_ACCR.groupby(by=['Quarter','Month','Week','COMPOUND','QA QUEUE NAME'], as_index = False).agg({'Output':'sum','Failed appeal':'sum','Edge case':'sum'})
TOTAL_ACCR_WEEK['%Accr'] = 1-(TOTAL_ACCR_WEEK['Failed appeal'] / TOTAL_ACCR_WEEK['Output'])
TOTAL_ACCR_WEEK['%Overall_Accr'] = 1-((TOTAL_ACCR_WEEK['Edge case']+TOTAL_ACCR_WEEK['Failed appeal']) / TOTAL_ACCR_WEEK['Output'])
TOTAL_ACCR_WEEK = TOTAL_ACCR_WEEK.dropna(subset=['%Accr'])
TOTAL_ACCR_WEEK = TOTAL_ACCR_WEEK[TOTAL_ACCR_WEEK['Week'].str.contains('|'.join(week_filter))]
TOTAL_ACCR_WEEK
print(1-(sum(TOTAL_ACCR_WEEK['Failed appeal'])/sum(TOTAL_ACCR_WEEK['Output'])))

0.9990667528879221


In [25]:
TOTAL_ACCR_WEEK

,Quarter,Month,Week,COMPOUND,QA QUEUE NAME,Output,Failed appeal,Edge case,%Accr,%Overall_Accr
0,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Recommendation L2L Sample QA,69.0,1.0,1.0,0.985507,0.971014
1,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search CB Sample QA,1.0,0.0,0.0,1.000000,1.000000
2,4.0,Nov,W43 (10.26 - 11.01),E-UPL,[VN] Mall UP Search L2L Sample QA,121.0,2.0,0.0,0.983471,0.983471
3,4.0,Nov,W43 (10.26 - 11.01),G-UPL,[VN] UP Livestream Sample QA,357.0,0.0,0.0,1.000000,1.000000
4,4.0,Nov,W43 (10.26 - 11.01),G-UPL,[VN] UP Video Sample QA,917.0,0.0,0.0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
222,4.0,Oct,W43 (10.26 - 11.01),G-UPL,[VN] UP Product Local Sample QA,624.0,0.0,0.0,1.000000,1.000000
223,4.0,Oct,W43 (10.26 - 11.01),Labeling,TC_HCM_MOD_MODEL LABELING_CCR,2149.0,9.0,0.0,0.995812,0.995812
224,4.0,Oct,W43 (10.26 - 11.01),Labeling,TC_HCM_MOD_MODEL LABELING_CONTENT,382.0,1.0,0.0,0.997382,0.997382
225,4.0,Oct,W43 (10.26 - 11.01),Labeling,TC_HCM_MOD_MODEL LABELING_IPR,1313.0,4.0,0.0,0.996954,0.996954


In [26]:
date_1 = '2023-11-01'
date_2 = '2023-11-12'

In [27]:
TOTAL_ACCR = pd.concat([Final_Mod_Accr, Final_E_UPL, Final_G_UPL, Final_Mod_Accr_sideproject, final_labeling], axis=0)
unique_weeks = TOTAL_ACCR['Week'].unique()
TOTAL_ACCR = TOTAL_ACCR[(TOTAL_ACCR['Date'].dt.strftime('%Y-%m-%d') >= date_1) & (TOTAL_ACCR['Date'].dt.strftime('%Y-%m-%d') <= date_2)]
TOTAL_ACCR = round(1 - TOTAL_ACCR['Failed appeal'].sum() / TOTAL_ACCR['Output'].sum(), 4)
print("Accr: " + str(date_1) + " --> " + str(date_2) + " : " + str(TOTAL_ACCR))
all_sheets_WBR = {'Completion Rate': CR_category,
                  'Input ouput qa':CR_rawdata,
                  'Regular Appeal':TOTAL_ACCR_WEEK,
                  'QA Accuracy': final_qa_accr,
                  'Gen UPL': Final_G_UPL,
                  'Shop UPL': Final_E_UPL,
                  'Policy Error':Policy_errors_count,
                  'RCA': RCA_diff_dive_count,
                  'FN_FP':All_fn_fp,
                  'FN_FP_CP':All_fn_fp_cp}

Accr: 2023-11-01 --> 2023-11-12 : 0.9991


In [28]:
WBR_writer = pd.ExcelWriter('Queue Accr.xlsx', engine='xlsxwriter')

for sheet_name in all_sheets_WBR.keys():
    all_sheets_WBR[sheet_name].to_excel(WBR_writer, sheet_name=sheet_name, index=True)

WBR_writer.close()